In [1]:
from matplotlib import pyplot as plt
# from astropy.utils.data import download_file
# from astropy.time import Time, TimeDelta
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time, TimeDelta
# import xml.etree.ElementTree as ET, gzip, io

from dateutil.parser import parse as date_parse
import numpy as np
import pandas as pd

from collections import namedtuple
from glob import glob
from astropy.io import fits
from astropy.table import Table

from tqdm import tqdm_notebook

from piaa import exoplanets
import exodata

In [2]:
#!gsutil ls gs://panoptes-survey/PAN006/**/**/**/*.fz > /var/panoptes/pan006.txt

In [3]:
# Load our list of stored images
fn = '/var/panoptes/pan006.txt'

fits_fn = !cat $fn | grep -v 'pointing'

In [4]:
transits = dict()
errors = dict()

for fn in tqdm_notebook(fits_fn):
    if 'pointing' in fn:
        continue 
    
    try:
        _, _, _, unit_id, field, cam_id, seq_id, img_id = fn.split('/')
    except ValueError:
        continue 
        
    if seq_id in transits:
        continue
        
    # Get the time from the image
    img_id = img_id.split('.')[0]
    try:
        t0 = Time(date_parse(img_id), format='datetime')
    except Exception as e:
        if str(e) not in errors:
            print(img_id, e)
            errors[str(e)] = True
        continue
        
    # Get exoplanet info
    try:
        planet = exoplanets.Exoplanet(field)
    except KeyError as e:
        if str(e) not in errors:
            print(e)
            errors[str(e)] = True        
        continue
        
    try:
        if planet.transit_duration.value == 0:
            k = '{}_duration'.format(planet.name)
            if k not in errors:
                print("No duration for", planet.name)
                errors[k] = True
            continue

        if planet.transit_midpoint.value == 0:
            k = '{}_midpoint'.format(planet.name)
            if k not in errors:
                print("No midpoint for", planet.name)
                errors[k] = True
            continue        

        if planet.period.value == 0:
            k = '{}_period'.format(planet.name)
            if k not in errors:
                print("No period for", planet.name)
                errors[k] = True
            continue                
    except Exception as e:
        if str(e) not in errors:
            print(e)
            errors[str(e)] = True                
    
    # Determine if in transit
    try:
        in_t = planet.in_transit(t0, with_times=True)
    except Exception as e:
        if str(e) not in errors:
            print(e)
            errors[str(e)] = True        
        continue    

    if in_t[0]:
        transits[seq_id] = {'transit': in_t[1], 'planet': planet}


'No matches found for key 55CncE'
'No matches found for key Epic211089792'
list index out of range
'No matches found for key Kic8462852'



In [ ]:
# Three subplots sharing both x/y axes

f, axes = plt.subplots(len(transits), sharey=True)
f.set_size_inches(12, 2 * len(transits))

for i, seq_id in enumerate(transits.keys()):

    try:
        ax = axes[i]
    except Exception:
        ax = axes
    
    t = transits[seq_id]['transit']
    planet = transits[seq_id]['planet']
    image_times = sorted(set(([x.split('/')[7].split('.')[0] for x in fits_fn if seq_id in x])))
                                
    phases = {planet.get_phase(Time(date_parse(t))):Time(date_parse(t)) for t in image_times}
    
    ax.scatter(list(phases.keys()), [1 for x in range(len(phases))], color='b', marker='.', label='Observation')
    
    print(t)
    print(planet.get_phase(t.ingress), planet.get_phase(t.egress))
    print()
    ax.axvline(planet.get_phase(t.ingress), 0, color='r', ls='--', alpha=0.5, label='ingress')
    ax.axvline(planet.get_phase(t.egress), 0, color='r', ls='--', alpha=0.5, label='egress')
        
    ax.legend()
        
    ax.set_title("{} {}".format(planet.info['pl_hostname'], seq_id))
    ax.set_xlim(-0.5, 0.5)
    ax.set_ylim(0.5, 1.5)
    ax.get_yaxis().set_ticks([])
    
    f.subplots_adjust(hspace=0)
    
f.savefig('/var/panoptes/pan006_matches.png')

Transit(ingress=<Time object: scale='utc' format='jd' value=2458152.7223013034>, midpoint=<Time object: scale='utc' format='jd' value=2458152.7710013036>, egress=<Time object: scale='utc' format='jd' value=2458152.8197013037>)
-0.015171300732212512 0.015142483131656104

Transit(ingress=<Time object: scale='utc' format='jd' value=2458140.5442152782>, midpoint=<Time object: scale='utc' format='jd' value=2458140.588565278>, egress=<Time object: scale='utc' format='jd' value=2458140.632915278>)
-0.01579190279455911 0.015767194811302294

Transit(ingress=<Time object: scale='utc' format='jd' value=2458162.7591652777>, midpoint=<Time object: scale='utc' format='jd' value=2458162.805315278>, egress=<Time object: scale='utc' format='jd' value=2458162.851465278>)
-0.03479468294962218 0.034742364652634534

Transit(ingress=<Time object: scale='utc' format='jd' value=2458162.7591652777>, midpoint=<Time object: scale='utc' format='jd' value=2458162.805315278>, egress=<Time object: scale='utc' format